In [1]:
import numpy as np
from PIL import Image
import csv
import os
from os import listdir
from os.path import isfile, join, splitext
import shutil
import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

import faiss

%matplotlib inline

In [3]:
# generate test np
d = 2                           # dimension
nb = 10                      # database size
nq = 1                     # nb of queries
np.random.seed(1234)             # make reproducible

In [4]:
xb = np.random.random((nb, d)).astype('float32')

In [5]:
xq = np.random.random((nq, d)).astype('float32')

In [6]:
# check np shape
print('xb.shape', xb.shape)
print('xq.shape', xq.shape)
print('xb[0]', xb[0])
print('xb[1]', xb[1])
#print('xb[99]', xb[99])

xb.shape (10, 2)
xq.shape (1, 2)
xb[0] [0.19151945 0.62210876]
xb[1] [0.43772775 0.7853586 ]


# 4096d - 1M dataset test on CPU machine

In [3]:
# generate test np
d = 4096                           # dimension
nb = 1000000                      # database size
nq = 1                     # nb of queries
np.random.seed(1234)             # make reproducible

In [4]:
%time xb1 = np.random.random((nb, d)).astype('float32')

CPU times: user 40.2 s, sys: 10 s, total: 50.3 s
Wall time: 50.3 s


In [5]:
%time xq1 = np.random.random((nq, d)).astype('float32')

CPU times: user 205 µs, sys: 49 µs, total: 254 µs
Wall time: 145 µs


In [6]:
# check np shape
print('xb1.shape', xb1.shape)
print('xq1.shape', xq1.shape)
print('xb1[0]', xb1[0])
print('xb1[1]', xb1[1])


xb1.shape (1000000, 4096)
xq1.shape (1, 4096)
xb1[0] [0.19151945 0.62210876 0.43772775 ... 0.9750933  0.9089286  0.8868944 ]
xb1[1] [0.9369065  0.95599407 0.38951334 ... 0.6071783  0.3248492  0.75957   ]


In [7]:
index = faiss.IndexFlatL2(d)   # build the index
print('index.is_trained : ', index.is_trained)

index.is_trained :  True


In [8]:
%time index.add(xb1)                  # add vectors to the index
print('index.ntotal : ', index.ntotal)

CPU times: user 2.78 s, sys: 3.12 s, total: 5.9 s
Wall time: 5.9 s
index.ntotal :  1000000


In [10]:
k = 100                          
%time D, I = index.search(xb1[:10], k)

CPU times: user 26.4 s, sys: 474 ms, total: 26.9 s
Wall time: 17.2 s


In [11]:
k = 1000                          
%time D, I = index.search(xb1[:10], k)

CPU times: user 26 s, sys: 573 ms, total: 26.6 s
Wall time: 17 s


In [13]:
k = 1000000                     
%time D, I = index.search(xb1[:10], k)

CPU times: user 28.7 s, sys: 506 ms, total: 29.2 s
Wall time: 19.6 s


# consume total 32G memory - 1M

# 4096d - 1M dataset - IndexIVFFlat - test on CPU machine
### https://github.com/facebookresearch/faiss/wiki/Faster-search

In [2]:
# generate test np
d = 4096                           # dimension
nb = 1000000                      # database size
nq = 1                     # nb of queries
np.random.seed(1234)             # make reproducible

In [3]:
%time xb2 = np.random.random((nb, d)).astype('float32')

CPU times: user 40.5 s, sys: 9.94 s, total: 50.4 s
Wall time: 50.4 s


In [4]:
%time xq2 = np.random.random((nq, d)).astype('float32')

CPU times: user 336 µs, sys: 80 µs, total: 416 µs
Wall time: 230 µs


In [7]:
# check np shape
print('xb2.shape', xb2.shape)
print('xq2.shape', xq2.shape)
print('xb2[0]', xb2[0])
print('xb2[1]', xb2[1])

xb2.shape (1000000, 4096)
xq2.shape (1, 4096)
xb2[0] [0.19151945 0.62210876 0.43772775 ... 0.9750933  0.9089286  0.8868944 ]
xb2[1] [0.9369065  0.95599407 0.38951334 ... 0.6071783  0.3248492  0.75957   ]


In [8]:
nlist = 100
%time quantizer = faiss.IndexFlatL2(d)  # the other index


CPU times: user 85 µs, sys: 0 ns, total: 85 µs
Wall time: 75.3 µs


In [9]:
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
       # here we specify METRIC_L2, by default it performs inner-product search
%time index.train(xb2)
print('index.is_trained', index.is_trained)


CPU times: user 13.1 s, sys: 110 ms, total: 13.2 s
Wall time: 9.13 s
index.is_trained True


In [10]:
%time index.add(xb2)                  # add may be a bit slower as well

CPU times: user 25.1 s, sys: 7.47 s, total: 32.5 s
Wall time: 18.2 s


In [11]:
k = 100
%time D, I = index.search(xq2, k)     # actual search
print(I[-5:])                  # neighbors of the 5 last queries

CPU times: user 280 ms, sys: 12.9 ms, total: 292 ms
Wall time: 49.2 ms
[[ 17516 996792 704912 176368 954419 494699 496710 517952 607979 979484
  105983 295458 449051 311091 526109 342930 195438 237684 582749 967920
  992917 602159 889923 254086 839513 627050 334330 198850 351587 655281
  343611 658278 176434 961115 884359  34328 440934 436925 273581 312648
  282765 744637 209712 735236 660071 210883 481372 857491 386408 436200
  472475 181153 639618 411730 570182 350001 635997  40831 417408 712217
  441586 972743 716412 161942 246741  75285 803035 447512 220153 298677
  203491 298138 836838  66162 581952 960145 590166 167672 171610 997327
  513954 335598 962371 100995 858014 216834   2967 198443 595122 999550
  961407 173950 133647 965664 851552 314198 713407 867221 250371 323723]]


In [16]:
k = 1000000
%time D, I = index.search(xq2, k)     # actual search
print(I[-5:])                  # neighbors of the 5 last queries

CPU times: user 1.27 s, sys: 34.3 ms, total: 1.31 s
Wall time: 442 ms
[[288749 715358  17516 ...     -1     -1     -1]]


In [15]:
k = 100
index.nprobe = 10              # default nprobe is 1, try a few more - getting slow
%time D, I = index.search(xq2, k)
print(I[-5:])                  # neighbors of the 5 last queries

CPU times: user 1.18 s, sys: 53.3 ms, total: 1.23 s
Wall time: 362 ms
[[288749 715358  17516 586587  81632 996792 530640 774878 952097  94876
  108618 308878 704912 924934 611590 230616 841005 245976 872831 780143
  539491 284597 988863 591019 407840  86768 850327 669668  55910 173758
  499498 176368 804002 823482 180233 311296 748047 856042 288396 657952
  390758 954419 494699 553111 496710 559531 847603 603161 121768 864780
  517952 216702 472374 607979 979484 424780 759982 575505 105983 295458
  556660 219738 449051 858834 614005  76661 471826 281163  82897 311091
  111533 522424 445932 526109 342930 254962 379530 994310 577652 462512
   87598 365283 736914 482052 370448 917392 195438 445633 683551 496163
   99154 237684  73447 112189 707413 582749 223062 552328 286381 776169]]


# 4096d - 1M dataset - IndexIVFPQ - test on CPU machine
### https://github.com/facebookresearch/faiss/wiki/Lower-memory-footprint

In [2]:
# generate test np
d = 4096                           # dimension
nb = 1000000                      # database size
nq = 1                     # nb of queries
np.random.seed(1234)             # make reproducible

In [3]:
%time xb3 = np.random.random((nb, d)).astype('float32')

CPU times: user 40.5 s, sys: 9.83 s, total: 50.4 s
Wall time: 50.4 s


In [14]:
%time xq3 = np.random.random((nq, d)).astype('float32')

CPU times: user 592 µs, sys: 0 ns, total: 592 µs
Wall time: 331 µs


In [15]:
# check np shape
print('xb3.shape', xb3.shape)
print('xq3.shape', xq3.shape)
print('xb3[0]', xb3[0])
print('xq3[0]', xq3[0])

xb3.shape (1000000, 4096)
xq3.shape (1, 4096)
xb3[0] [0.19151945 0.62210876 0.43772775 ... 0.9750933  0.9089286  0.8868944 ]
xq3[0] [0.01002005 0.54112315 0.66451573 ... 0.6636337  0.47418234 0.8247013 ]


In [16]:
nlist = 100
m = 8    # number of subquantizers

%time quantizer = faiss.IndexFlatL2(d)  # the other index


CPU times: user 48 µs, sys: 0 ns, total: 48 µs
Wall time: 39.3 µs


In [17]:
%time index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)  # 8 specifies that each sub-vector is encoded as 8 bits

CPU times: user 1.41 ms, sys: 0 ns, total: 1.41 ms
Wall time: 714 µs


In [18]:
%time index.train(xb3)

CPU times: user 1min 22s, sys: 1.05 s, total: 1min 24s
Wall time: 21.4 s


In [23]:
%time index.add(xb3)

CPU times: user 1min 27s, sys: 6.48 s, total: 1min 33s
Wall time: 15.7 s


In [24]:
k = 100
%time D, I = index.search(xq3, k)     # actual search
print(I[-5:])                  # neighbors of the 5 last queries

CPU times: user 15.8 ms, sys: 0 ns, total: 15.8 ms
Wall time: 2.69 ms
[[950785 820669 326553 435721  65112 790912 252108 297277 950015 709121
  135308 902144  44375 109113 237060 986685 651956 810994 565901 474236
  393355 829021 535464 673586 422044 238157 226394 991735 938248 491356
  530970 535864 241677 548135 657035 232535 715748 449902 469741 138512
  749170 879483 955599 397868 889253 519623 922312 260403 589513 873843
  527506 784739 621625  41708 342174  84243 141112 443282 156896 857979
  217169 690689 587435 812108 362872 658326 564804 532177 681190 175824
  233486 709392 235473  55291 489019 412045 236827 665792 659779 202736
  997813 180261 570520 902677 369688  38802 845289 242955 651508 362905
  689602 295005 151699 267795 834152 103529 722777 581416  64784 561154]]


In [25]:
k = 1000000
%time D, I = index.search(xq3, k)     # actual search
print(I[-5:])                  # neighbors of the 5 last queries

CPU times: user 486 ms, sys: 22.8 ms, total: 509 ms
Wall time: 85 ms
[[950785 820669 326553 ...     -1     -1     -1]]


In [26]:
k = 100
index.nprobe = 10              # default nprobe is 1, try a few more - getting slow
%time D, I = index.search(xq3, k)
print(I[-5:])                  # neighbors of the 5 last queries

CPU times: user 10.9 ms, sys: 3.44 ms, total: 14.4 ms
Wall time: 2.45 ms
[[950785 820669 326553 435721  65112 790912 252108 297277 950015 709121
  135308 902144  44375 109113 237060 986685 651956 810994 565901 474236
  393355 829021 535464 673586 422044 238157 226394 991735 938248 491356
  530970 535864 241677 548135 657035 232535 715748 449902 469741 138512
  749170 879483 955599 397868 889253 519623 922312 260403 589513 873843
  527506 784739 621625  41708 342174  84243 141112 443282 156896 857979
  217169 690689 587435 812108 362872 658326 564804 532177 681190 175824
  233486 709392 235473  55291 489019 412045 236827 665792 659779 202736
  997813 180261 570520 902677 369688  38802 845289 242955 651508 362905
  689602 295005 151699 267795 834152 103529 722777 581416  64784 561154]]


# 4096d - 4M dataset - IndexIVFPQ - test on CPU machine

In [2]:
# generate test np
d = 4096                           # dimension
nb = 4000000                      # database size
nq = 1                     # nb of queries
np.random.seed(1234)             # make reproducible

In [3]:
%time xb1 = np.random.random((nb, d)).astype('float32')

CPU times: user 2min 46s, sys: 53.3 s, total: 3min 39s
Wall time: 3min 39s


In [4]:
%time xq1 = np.random.random((nq, d)).astype('float32')

CPU times: user 510 µs, sys: 163 µs, total: 673 µs
Wall time: 476 µs


In [5]:
# check np shape
print('xb1.shape', xb1.shape)
print('xq1.shape', xq1.shape)
print('xb1[0]', xb1[0])
print('xb1[1]', xb1[1])


xb1.shape (4000000, 4096)
xq1.shape (1, 4096)
xb1[0] [0.19151945 0.62210876 0.43772775 ... 0.9750933  0.9089286  0.8868944 ]
xb1[1] [0.9369065  0.95599407 0.38951334 ... 0.6071783  0.3248492  0.75957   ]


In [6]:
%time index = faiss.IndexFlatL2(d)   # build the index
print('index.is_trained : ', index.is_trained)

index.is_trained :  True


In [7]:
%time index.add(xb1)                  # add vectors to the index
print('index.ntotal : ', index.ntotal)

CPU times: user 12.3 s, sys: 12.5 s, total: 24.8 s
Wall time: 24.8 s
index.ntotal :  4000000


In [8]:
k = 100                          
%time D, I = index.search(xb1[:10], k)

CPU times: user 1min 49s, sys: 2.59 s, total: 1min 51s
Wall time: 1min 8s


In [9]:
k = 1000                          
%time D, I = index.search(xb1[:10], k)

CPU times: user 1min 48s, sys: 2.55 s, total: 1min 51s
Wall time: 1min 8s


In [10]:
k = 4000000                     
%time D, I = index.search(xb1[:10], k)

CPU times: user 1min 57s, sys: 2.65 s, total: 2min
Wall time: 1min 19s


# consume total 124G memory - 4M

# 4096d - 4M dataset - IndexIVFFlat - test on CPU machine

In [12]:
index = None

In [13]:
nlist = 100
%time quantizer = faiss.IndexFlatL2(d)  # the other index

CPU times: user 94 µs, sys: 0 ns, total: 94 µs
Wall time: 90.4 µs


In [14]:
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
       # here we specify METRIC_L2, by default it performs inner-product search
%time index.train(xb1)
print('index.is_trained', index.is_trained)

CPU times: user 1min 1s, sys: 55.6 s, total: 1min 57s
Wall time: 1min 40s
index.is_trained True


In [15]:
%time index.add(xb1)                  # add may be a bit slower as well

CPU times: user 2min 31s, sys: 2min 2s, total: 4min 34s
Wall time: 1min 31s


In [19]:
k = 100
%time D, I = index.search(xq1, k)     # actual search
print(I[-5:])                  # neighbors of the 5 last queries

CPU times: user 5.46 s, sys: 304 ms, total: 5.76 s
Wall time: 2.23 s
[[3084379  635044 2664931 3112684 1710949  893567 1094423  372070 2866383
  1202304  209481 1398511 3844585 2981741  287530 2470413  611851  392155
  2906991    6424 2589890 3048104  519337 1704052 2548378 2116930 1004419
  1468472  868104 1682308 3918233  580660  852756 2422099 1210681 3133462
  2971871 3375968 2295913 1952157 1156852 1868846 1077897  315169 3739974
  3036041 2734053 2224222 1203557 2494437 1352655 2319937 1413803 2457798
  3149634 3024316 1316542 1615482 1100538 3463449 2627077 1513193 2508481
  3496922 3515392  488576  973571 2608497  972546 3084535 1205286  223328
   847236  973642  839132 1544645  926482 3421090 3681122 1937255 2823441
  3459752 3229873 3659407 3269189 1290480 1233174 3245192  906425 1216803
  2200457 2676142 2218092 3614159 1617936 2651628  876088 3199565 1598095
  2422860]]


In [20]:
k = 4000000
%time D, I = index.search(xq1, k)     # actual search
print(I[-5:])                  # neighbors of the 5 last queries

CPU times: user 5.39 s, sys: 202 ms, total: 5.6 s
Wall time: 2.6 s
[[3084379  635044 2664931 ...      -1      -1      -1]]


In [21]:
k = 100
index.nprobe = 10              # default nprobe is 1, try a few more - getting slow
%time D, I = index.search(xq1, k)
print(I[-5:])                  # neighbors of the 5 last queries

CPU times: user 5.46 s, sys: 248 ms, total: 5.7 s
Wall time: 2.12 s
[[3084379  635044 2664931 3112684 1710949  893567 1094423  372070 2866383
  1202304  209481 1398511 3844585 2981741  287530 2470413  611851  392155
  2906991    6424 2589890 3048104  519337 1704052 2548378 2116930 1004419
  1468472  868104 1682308 3918233  580660  852756 2422099 1210681 3133462
  2971871 3375968 2295913 1952157 1156852 1868846 1077897  315169 3739974
  3036041 2734053 2224222 1203557 2494437 1352655 2319937 1413803 2457798
  3149634 3024316 1316542 1615482 1100538 3463449 2627077 1513193 2508481
  3496922 3515392  488576  973571 2608497  972546 3084535 1205286  223328
   847236  973642  839132 1544645  926482 3421090 3681122 1937255 2823441
  3459752 3229873 3659407 3269189 1290480 1233174 3245192  906425 1216803
  2200457 2676142 2218092 3614159 1617936 2651628  876088 3199565 1598095
  2422860]]


# 4096d - 4M dataset - IndexIVFPQ - test on CPU machine

In [22]:
index = None

In [23]:
nlist = 100
m = 8    # number of subquantizers

%time quantizer = faiss.IndexFlatL2(d)  # the other index

CPU times: user 0 ns, sys: 77 µs, total: 77 µs
Wall time: 70.6 µs


In [24]:
%time index = faiss.IndexIVFPQ(quantizer, d, nlist, m, 8)  # 8 specifies that each sub-vector is encoded as 8 bits

CPU times: user 2.24 ms, sys: 336 µs, total: 2.57 ms
Wall time: 48.9 ms


In [25]:
%time index.train(xb1)

CPU times: user 3min 7s, sys: 10.1 s, total: 3min 17s
Wall time: 53 s


In [26]:
%time index.add(xb1)

CPU times: user 15min 36s, sys: 1min 53s, total: 17min 30s
Wall time: 45.6 s


In [27]:
k = 100
%time D, I = index.search(xq1, k)     # actual search
print(I[-5:])                  # neighbors of the 5 last queries

CPU times: user 25.4 ms, sys: 1.51 ms, total: 27 ms
Wall time: 1.6 ms
[[ 127409 2342266 1037896 1521163 1393329 2439532 2299464 1534446 3372148
  1970457 3978389  778330 1504218  835479 3486445  987769 2475158  893435
  2697521 3359094 3776263 2796849 2581001 2788703 2967623 1228360 1905467
   291850 2847726  368892 1988905 3472724 1638806 1245538 2913663 3790735
  2422512 3510363 1697489 3606340 3857401 3509099 3834708  742257  184484
  1297577 1567648 1826241 3065655 2250837 1544324 3558343 2780172  874134
  3701899  395462 3215684 1672258 3456573 2421592 3253209  406453 1027972
  2506054 3681487 3124800 3129955 1126289 2403071 2540322 2820147 3152535
  3316044 1266132 1412230 3184104  340067 3590882 2266437 3716270  146135
   725439  317242 1751432 1502555 2980895 2908823  969731  382774 3439147
  3040912  132119  356195 3322525 1745964 2531157  273240 1843714 3361213
  2204547]]


In [28]:
k = 4000000
%time D, I = index.search(xq1, k)     # actual search
print(I[-5:])                  # neighbors of the 5 last queries

CPU times: user 3.82 s, sys: 162 ms, total: 3.98 s
Wall time: 176 ms
[[ 127409 2342266 1037896 ...      -1      -1      -1]]


In [29]:
k = 100
index.nprobe = 10              # default nprobe is 1, try a few more - getting slow
%time D, I = index.search(xq1, k)
print(I[-5:])                  # neighbors of the 5 last queries

CPU times: user 453 ms, sys: 25 ms, total: 478 ms
Wall time: 20.1 ms
[[ 829986  606382 1238934  127409 3887834 3281491 3338736 3247951 3881079
  2738662 3656462 2342266 1395944 1753345 3138655 1037896 1156135 3439174
  2881988 2468678 1769993 3262899   53189 2947183 3713404 1521163 1393329
   253935 1694870 3184420 3335537 2214994  221952  362559 2128623  523339
  1105613  731038 2372959 3427204  950106 1057591 3781549 2439532 2299464
  1060821 1534446 1744762 1796497 2084371 3865922  735461 3368827 3122146
  1070554 3682704  496433 1198678 1801430 2624039  645067 3372148 1095576
   971787 1999256  770126 3390214 3329901 3142749  685005  199908  687829
   520091  377197 1248020 2520758 2193186 2824952 2625454 2056869  155162
  1659685 1970457 1879888 1491207 2124764  634829 3375757 1989306 2480441
  2369585 2610616 1597392  410207 3246157 3196901 3978389  204538 2199991
  2274433]]


# consume total 63G memory - 4M